# בנייה עם משפחת המודלים של Meta

## הקדמה

בשיעור זה נעסוק ב:

- היכרות עם שני המודלים המרכזיים ממשפחת Meta – Llama 3.1 ו-Llama 3.2
- הבנת מקרי השימוש והתרחישים המתאימים לכל מודל
- דוגמת קוד המדגימה את היכולות הייחודיות של כל מודל

## משפחת המודלים של Meta

בשיעור זה נחקור שני מודלים ממשפחת Meta, או "עדר ה-Llama" – Llama 3.1 ו-Llama 3.2

למודלים אלו יש גרסאות שונות, והם זמינים בשוק המודלים של Github. תוכלו למצוא מידע נוסף על שימוש במודלים של Github ל[פיתוח אב-טיפוס עם מודלים של בינה מלאכותית](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

גרסאות המודלים:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*הערה: Llama 3 זמין גם הוא במודלים של Github, אך לא יידון בשיעור זה*


## Llama 3.1

עם 405 מיליארד פרמטרים, Llama 3.1 נכנס לקטגוריית מודלי השפה הפתוחים (LLM) בקוד פתוח.

הגרסה הזו היא שדרוג לגרסה הקודמת Llama 3 ומציעה:

- חלון הקשר גדול יותר – 128k טוקנים לעומת 8k טוקנים
- כמות מקסימלית גדולה יותר של טוקנים לפלט – 4096 לעומת 2048
- תמיכה רב-לשונית משופרת – בזכות הגדלת כמות הטוקנים באימון

היכולות האלו מאפשרות ל-Llama 3.1 להתמודד עם מקרי שימוש מורכבים יותר בפיתוח יישומי GenAI, כולל:
- קריאה לפונקציות באופן מובנה – היכולת להפעיל כלים ופונקציות חיצוניות מחוץ לזרימת העבודה של ה-LLM
- ביצועי RAG טובים יותר – בזכות חלון הקשר הרחב יותר
- יצירת נתונים סינתטיים – היכולת ליצור נתונים יעילים למשימות כמו כיוונון עדין


### קריאה לפונקציות מובנות

Llama 3.1 עברה כיוונון נוסף כדי להיות יעילה יותר בביצוע קריאות לפונקציות או כלים. בנוסף, יש לה שני כלים מובנים שהמודל יודע לזהות מתי יש צורך להשתמש בהם לפי הבקשה של המשתמש. הכלים הם:

- **Brave Search** - ניתן להשתמש בו כדי לקבל מידע עדכני כמו מזג האוויר על ידי חיפוש באינטרנט
- **Wolfram Alpha** - ניתן להשתמש בו לחישובים מתמטיים מורכבים יותר, כך שאין צורך לכתוב פונקציות משלך.

ניתן גם ליצור כלים מותאמים אישית שה-LLM יוכל לקרוא להם.

בדוגמה הקודמת למטה:

- אנחנו מגדירים את הכלים הזמינים (brave_search, wolfram_alpha) בהנחיית המערכת.
- שולחים בקשה מהמשתמש ששואלת על מזג האוויר בעיר מסוימת.
- ה-LLM יגיב בקריאה לכלי Brave Search, שתיראה כך: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*הערה: בדוגמה זו מתבצעת רק קריאה לכלי. אם תרצה לקבל את התוצאות, תצטרך ליצור חשבון חינמי בדף ה-API של Brave ולהגדיר את הפונקציה בעצמך.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

למרות ש-Llama 3.1 הוא מודל שפה גדול, אחת המגבלות שלו היא חוסר תמיכה במולטימודאליות. כלומר, היכולת להשתמש בסוגי קלט שונים כמו תמונות כהנחיה ולקבל תגובות בהתאם. היכולת הזו היא אחת התכונות המרכזיות של Llama 3.2. תכונות נוספות כוללות:

- מולטימודאליות – מסוגל להעריך גם הנחיות טקסט וגם תמונה
- וריאציות בגודל קטן עד בינוני (11B ו-90B) – מאפשר אפשרויות פריסה גמישות,
- וריאציות טקסט בלבד (1B ו-3B) – מאפשרות להריץ את המודל על מכשירים ניידים/קצה ומספקות השהיה נמוכה

התמיכה במולטימודאליות מהווה צעד משמעותי בעולם המודלים בקוד פתוח. בדוגמה הבאה, הקוד מקבל גם תמונה וגם הנחיית טקסט כדי לקבל ניתוח של התמונה מ-Llama 3.2 90B.

### תמיכה מולטימודאלית עם Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## הלמידה לא מסתיימת כאן, המשיכו במסע

לאחר שסיימתם את השיעור הזה, בקרו ב-[אוסף הלמידה של בינה מלאכותית גנרטיבית](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) כדי להמשיך להעמיק את הידע שלכם בבינה מלאכותית גנרטיבית!



---

Certainly! Here is your text translated into Hebrew:

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור ייחשב כמקור הסמכותי. למידע קריטי, מומלץ לפנות לתרגום מקצועי על ידי אדם. איננו אחראים לכל אי-הבנה או פרשנות שגויה הנובעת מהשימוש בתרגום זה.
